# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/ahmedmansour/Desktop/datacourse/python-api-challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ilulissat,69.2167,-51.1000,-0.99,55,100,6.69,GL,1728833084
1,1,grytviken,-54.2811,-36.5092,0.04,74,25,4.88,GS,1728833363
2,2,al bawiti,28.3492,28.8659,33.74,20,0,4.40,EG,1728833364
3,3,ushuaia,-54.8000,-68.3000,10.81,54,0,5.14,AR,1728833124
4,4,tshabong,-26.0500,22.4500,37.49,6,6,1.26,BW,1728833367


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
%%capture --no-display

# Configure the map plot
city_data_map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.4,
    color = 'Country'
)
# Display the map
city_data_map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df['Max Temp']<27,:]
city_temp = city_temp0.loc[city_temp0['Max Temp']>21,:]
city_wind = city_temp.loc[city_temp['Wind Speed']<4.5,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]

# Drop any rows with null values
city_criteria = city_temp0.dropna()
city_criteria = city_temp.dropna()
city_criteria = city_wind.dropna()
city_criteria = city_cloud.dropna()


# Display sample data
city_criteria

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
59,59,guerrero negro,27.9769,-114.0611,22.91,65,0,2.76,MX,1728833441
73,73,tura,25.5198,90.2201,21.91,89,0,1.68,IN,1728833461
79,79,midland,31.9974,-102.0779,25.91,22,0,4.12,US,1728833471
139,139,yangjiang,21.8500,111.9667,23.55,85,0,2.06,CN,1728833557
154,154,garcia,25.8167,-100.5833,24.37,51,0,2.13,MX,1728833576
238,238,ivins,37.1686,-113.6794,22.85,22,0,1.78,US,1728833690
260,260,santiago,-33.4569,-70.6483,22.70,57,0,2.06,CL,1728833709
363,363,knoxville,35.9606,-83.9207,22.36,67,0,3.09,US,1728833623
368,368,antonio enes,-16.2325,39.9086,25.64,77,0,3.23,MZ,1728833875
406,406,morehead city,34.7229,-76.7260,24.47,58,0,3.09,US,1728833927


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_criteria.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
59,guerrero negro,MX,27.9769,-114.0611,65,
73,tura,IN,25.5198,90.2201,89,
79,midland,US,31.9974,-102.0779,22,
139,yangjiang,CN,21.8500,111.9667,85,
154,garcia,MX,25.8167,-100.5833,51,
238,ivins,US,37.1686,-113.6794,22,
260,santiago,CL,-33.4569,-70.6483,57,
363,knoxville,US,35.9606,-83.9207,67,
368,antonio enes,MZ,-16.2325,39.9086,77,
406,morehead city,US,34.7229,-76.7260,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
    "limit":limit 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
guerrero negro - nearest hotel: Plaza sal paraiso
tura - nearest hotel: Hotel Polo Orchid
midland - nearest hotel: DoubleTree by Hilton Hotel Midland Plaza
yangjiang - nearest hotel: Jinghu Hotel
garcia - nearest hotel: No hotel found
ivins - nearest hotel: Crescent Moon Inn
santiago - nearest hotel: apart arturo prat
knoxville - nearest hotel: Hilton Knoxville
antonio enes - nearest hotel: Hotel Quirimbas
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
woodlake - nearest hotel: Sleep Inn
pikeville - nearest hotel: Hampton Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
59,guerrero negro,MX,27.9769,-114.0611,65,Plaza sal paraiso
73,tura,IN,25.5198,90.2201,89,Hotel Polo Orchid
79,midland,US,31.9974,-102.0779,22,DoubleTree by Hilton Hotel Midland Plaza
139,yangjiang,CN,21.8500,111.9667,85,Jinghu Hotel
154,garcia,MX,25.8167,-100.5833,51,No hotel found
238,ivins,US,37.1686,-113.6794,22,Crescent Moon Inn
260,santiago,CL,-33.4569,-70.6483,57,apart arturo prat
363,knoxville,US,35.9606,-83.9207,67,Hilton Knoxville
368,antonio enes,MZ,-16.2325,39.9086,77,Hotel Quirimbas
406,morehead city,US,34.7229,-76.7260,58,"Bask Hotel Big Rock Landing, Trademark Collect..."


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 600,
    size = "Humidity",
    scale =0.9,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)